In [1]:
import numpy as np
import pandas as pd

In [2]:
def to_dict(line):
    line = line.split()
    line_dict = {}
    line_dict['ix'], line_dict['iy'], line_dict['icloudy'], line_dict['icol'], line_dict['irow'], \
    line_dict['x'], line_dict['y'], line_dict['MASL'], line_dict['land_percent'], line_dict['nwl'] = line[:10]
    
    wl_num = int(line_dict['nwl'])
    
    line_dict['wl'] = line[10:10 + wl_num]
    line_dict['nip'] = line[10 + wl_num: 10 + 2 * wl_num]
    
    chanel_num = sum([int(el) for el in line_dict['nip']])
    
    
    last = 10 + 2 * int(line_dict['nwl']) + chanel_num
    line_dict['meas_type'] = line[22: last]
    
    line_dict['nbvm'] = line[last: last + chanel_num]
    last = last + chanel_num
    
    
    meas_num = sum([int(el) for el in line_dict['nbvm']])

    line_dict['sza'] = line[last: last + wl_num]
    last = last + wl_num

    line_dict['vza'] = line[last: last + meas_num]
    last = last + meas_num

    line_dict['raa'] = line[last: last + meas_num]
    last = last + meas_num

    line_dict['meas'] = line[last: last + meas_num]
    last = last + meas_num

    line_dict['ifcov'] = line[last: last + chanel_num]
    last = last + chanel_num 

    line_dict['ifmp'] = line[last: last + chanel_num]
    last = last + chanel_num 
    return line_dict

In [3]:
def get_data(line_dict):
    wl_num = int(line_dict['nwl'])
    chanel_num = sum([int(el) for el in line_dict['nip']])
    meas_num = sum([int(el) for el in line_dict['nbvm']])
    
    def mult(x):
        if type(x) == list:

            if len(x) == wl_num:
                l = [[a]*int(b) for a,b in zip(x,line_dict['nip'])]
                x = [item for sublist in l for item in sublist]

            if len(x) == chanel_num:
                l = [[a]*int(b) for a,b in zip(x,line_dict['nbvm'])]
                x =  [item for sublist in l for item in sublist]

            return x
        return [x] * meas_num

    t = pd.Series(line_dict).apply(mult)
    return pd.DataFrame(t.values.tolist(), index = t.index).T

In [4]:
f = open("/home/ihor/work_space/aerosol_data_processing/example_polder.sdat", "r")
text = f.readlines() 

In [5]:
data_list = []
for line in text:
    if len(line) == 103:
        cell_header = {}
        cell_header['NPIXELS'], cell_header['TIMESTAMP'], cell_header['HEIGHT_OBS'], \
        cell_header['NSURF'], cell_header['IFGAS'] = line.split()[:5]
    elif len(line) > 200:
        line_dict = to_dict(line)
        line_dict.update(cell_header)
        data = get_data(line_dict)
        data_list.append(data)
        

In [6]:
data = pd.concat(data_list)

In [49]:
line_dict = to_dict(line)